In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn as nn
from PIL import Image 
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchsummary import summary

import os

In [ ]:
%matplotlib inline
plt.style.use("ggplot")


class CFG:
    batch_size = 128
    num_epochs = 300
    workers = 4
    seed = 2021
    image_W = 64
    image_H = 64
    image_C = 3 ## number chanels  ngf = 64 #Size of feature maps in generator
    download = True
    dataroot = "data"
    nz = 100 #latent random input vector
    ngf = 64 #Size of feature maps in generator
    ndf = 64 #Size of feature maps in discriminator
    lr = 0.0002
    device = 'cuda'  
    sample_dir = "./images/"
    
if not os.path.exists(CFG.sample_dir):
    os.makedirs(CFG.sample_dir)
dataset = torchvision.datasets.CIFAR10(root=CFG.dataroot, download=CFG.download,
                               transform=transforms.Compose([
                                  transforms.Resize([CFG.image_H, CFG.image_W]),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))


In [ ]:
dataset[0][0].shape

In [ ]:
# denormalization image from range (-1)-1 to range 0-1 to display it
def unnomalization(x):
    batch_size_ = x.size(0)
    x = x.view(batch_size_ , -1)
    x -= x.min(1, keepdim=True)[0]
    x /= x.max(1, keepdim=True)[0]
    x = x.view(batch_size_, CFG.image_C, CFG.image_H, CFG.image_W)
    return x

img, label = dataset[0]
img = unnomalization(img.unsqueeze(0)).squeeze(0) # The functions expects first dim to be batch_size
print("Min: ", img.min().item())
print("Max: ", img.max().item())

In [ ]:
# show image sample with matplotlib
def plot_torch_image(img, **kwargs):
    '''
    Input image is a torch tensor with the following dims (C,H,W)
    To plot it with matplotlib, we need to change it to (H,W,C) 
    kwargs varaible is used to pass other parameters to 'imshow' function.
    '''
    plt.imshow(img.permute(1, 2, 0) , **kwargs)
    
plot_torch_image(img)

In [ ]:
# Create the dataloader
data_loader = DataLoader(dataset, batch_size=CFG.batch_size, shuffle=True, drop_last=True)


## GAN 
состоит из двух глубоких сетей, генератора и дискриминатора. генератор создает изображения, прежде чем научиться его обучать. Поскольку дискриминатор представляет собой модель бинарной классификации, мы можем использовать функцию потери бинарной кросс-энтропии для количественной оценки того, насколько хорошо он может различать реальные и сгенерированные изображения.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Generator(nn.Module):
    def __init__(self, nc, h, w, nz, ngf):
        #ConvTranspose2d - BatchNorm - Relu -ConvTranspose2d - BatchNorm - Relu -ConvTranspose2d - BatchNorm - Relu 
        #ConvTranspose2d - BatchNorm - Relu - ConvTranspose2d - Tanh
        
        # ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding 
        super(Generator, self).__init__()
        self.nc = nc
        self.h = h
        self.w = w
        self.ngf = ngf
        
        self.linear = nn.Linear(nz, nc * h * w)
        self.conv1 = nn.ConvTranspose2d(in_channels=nc, out_channels=ngf, kernel_size=(3,3), stride=1, padding=1)
        self.batch1 = nn.BatchNorm2d(ngf)
        self.conv2 = nn.ConvTranspose2d(in_channels=ngf, out_channels=ngf, kernel_size=(3,3), stride=1, padding=1)
        self.batch2 = nn.BatchNorm2d(ngf)
        self.conv3 = nn.ConvTranspose2d(in_channels=ngf, out_channels=ngf, kernel_size=(3,3), stride=1, padding=1)
        self.batch3 = nn.BatchNorm2d(ngf)
        self.conv4 = nn.ConvTranspose2d(in_channels=ngf, out_channels=ngf, kernel_size=(3,3), stride=1, padding=1)
        self.batch4 = nn.BatchNorm2d(ngf) 
        self.conv5 = nn.ConvTranspose2d(in_channels=ngf, out_channels=nc, kernel_size=(3,3), stride=1, padding=1)
        
    def forward(self, x):
        bs= x.shape[0]
        x = self.linear(x)
        x = x.view(bs, self.nc, self.h, self.w)
        x = self.conv1(x)
        x = self.batch1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = nn.ReLU()(x)     
        x = self.conv3(x)
        x = self.batch3(x)
        x = nn.ReLU()(x) 
        x = self.conv4(x)
        x = self.batch4(x)
        x = nn.ReLU()(x)
        x = self.conv5(x)
        return torch.tanh(x)


class Discriminator(nn.Module):
    def __init__(self, nc, h, w, ndf):
    #conv2d - leaky - conv2d - batchnorm - leaky - conv2d - batchnorm - leaky - conv - batchnorm - leaky - conv2d
        super(Discriminator, self).__init__()
        self.conv1 = nn.ConvTranspose2d(in_channels=nc, out_channels=ndf, kernel_size=(3,3), stride=1, padding=1)
        self.l_relu1 = nn.LeakyReLU(0.1)
        self.conv2 = nn.ConvTranspose2d(in_channels=ndf, out_channels=ndf, kernel_size=(3,3), stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ndf)
        self.l_relu2 = nn.LeakyReLU(0.1)
        self.conv3 = nn.ConvTranspose2d(in_channels=ndf, out_channels=ndf, kernel_size=(3,3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ndf)
        self.l_relu3 = nn.LeakyReLU(0.1)
        self.conv4 = nn.ConvTranspose2d(in_channels=ndf, out_channels=ndf, kernel_size=(3,3), stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(ndf)
        self.l_relu4 = nn.LeakyReLU(0.1)
        self.conv5 = nn.ConvTranspose2d(in_channels=ndf, out_channels=ndf, kernel_size=(3,3), stride=1, padding=1)
        
        self.linear = nn.Linear(262144, 1)
    def forward(self, x):
#         print("fwd: ", x.shape)
        x = self.conv1(x)
        x = self.l_relu1(x)
#         print("fwd: ", x.shape)
        x = self.conv2(x)
        x = self.bn1(x)
        x = self.l_relu2(x)
#         print("fwd: ", x.shape)
        x = self.conv3(x)
        x = self.bn2(x) 
        x = self.l_relu3(x)
#         print("fwd: ", x.shape)
        x = self.conv4(x)
        x = self.bn3(x) 
        x = self.l_relu4(x)
#         print("fwd: ", x.shape)
        x = self.conv5(x)
#         print("fwd: ", x.shape)
        x = torch.flatten(x, start_dim=1)
#         print("fwd: ", x.shape)
        x = self.linear(x)
#         print("fwd: ", x.shape)
        return nn.Sigmoid()(x)


In [ ]:
# create new Generator model
G = Generator(CFG.image_C, CFG.image_H, CFG.image_W, CFG.nz, CFG.ngf)
# create new Discriminator model
D = Discriminator(CFG.image_C, CFG.image_H, CFG.image_W, CFG.ndf)

In [ ]:
# show the output of model 
y = G(torch.randn(2, CFG.nz))
with torch.no_grad():
    for img in y:
        img= img.reshape(CFG.image_C, CFG.image_H, CFG.image_W)
        img = unnomalization(img.unsqueeze(0)).squeeze(0)
        plt.figure()
        plot_torch_image(img.detach())

In [ ]:
# define the criterion is nn.BCELoss()
loss_fn = nn.BCELoss()
## Define the optimizer for generator and discrimator
G_optim = torch.optim.Adam(G.parameters(), lr=CFG.lr)
D_optim = torch.optim.Adam(D.parameters(), lr=CFG.lr)

In [ ]:
def reset_grad():
    ## reset gradient for optimizer of generator and discrimator
    G_optim.zero_grad()
    D_optim.zero_grad()

def train_discriminator(images):
  
    # Create the labels which are later used as input for the BCE loss
    real_labels = torch.ones(CFG.batch_size, 1).to(CFG.device)
    fake_labels = torch.zeros(CFG.batch_size, 1).to(CFG.device)
    
    outputs = D(images)
    # Loss for real images
    loss_r = loss_fn(outputs, real_labels)
    real_score = outputs

    # Loss for fake images
    z = torch.randn(CFG.batch_size, CFG.nz).to(CFG.device)
    fake_images = G(z)
    outputs = D(fake_images)
    loss_f = loss_fn(outputs, fake_labels)
    fake_score = outputs
    # Sum losses
    d_loss = loss_r + loss_f
    # Adjust the parameters using backprop
    d_loss.backward()
    # Compute gradients
    D_optim.step()
    # Reset gradients
    reset_grad()

    return d_loss, real_score, fake_score

In [ ]:
def train_generator():
    # Generate fake images and calculate loss
    # z = torch.randn(batch_size, latent_size).to(device)
    z = torch.Tensor(np.random.normal(0, 1, (CFG.batch_size, CFG.nz))).to(CFG.device)
    fake_images = G(z)
    labels = torch.ones(CFG.batch_size, 1).to(CFG.device)
    # calculate the generator loss
    outputs = D(fake_images)
    g_loss = loss_fn(outputs, labels)
    # Backprop and optimize
    g_loss.backward()
    G_optim.step()
    # Reset gradients
    reset_grad()
    return g_loss, fake_images

## Start the training proccess

In [ ]:
def save_fake_images(index):
    sample_vectors = torch.randn(CFG.batch_size, CFG.nz , 1 , 1).to(CFG.device)
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 3, 64, 64)
    fake_fname = 'fake_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(unnomalization(fake_images), os.path.join(CFG.sample_dir, fake_fname), nrow=10)

In [ ]:
%%time
total_step = len(data_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []
G.to(CFG.device)
D.to(CFG.device)
for epoch in range(CFG.num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # Load a batch & transform to vectors
#         images = images.reshape(CFG.batch_size, -1).to(CFG.device)
        images = images.to(CFG.device)
        # Train the discriminator  
        d_loss, real_score, fake_score = train_discriminator(images)
        # Train the generator
        g_loss, fake_images = train_generator()
        # Inspect the losses
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, CFG.num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    # Sample and save images
    save_fake_images(epoch+1)

In [ ]:
Image(os.path.join(CFG.sample_dir, 'fake_images-0225.png'))

In [ ]:
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real Score', 'Fake score'])
plt.title('Scores');

In [ ]:
if not os.path.exists("./checkpoint/"):
    os.makedirs("./checkpoint/")
torch.save(
    self.G.state_dict(),
    "checkpoint/generator_model.pth")
torch.save(
    self.D.state_dict(),
    "checkpoint/discrimator_model.pth")